In [1]:
%%cmd
pip install pandas

Microsoft Windows [Version 10.0.26100.6899]
(c) Microsoft Corporation. All rights reserved.

C:\Users\alfred\DWH\shopzada>pip install pandas

C:\Users\alfred\DWH\shopzada>

In [3]:
import pandas as pd
import glob
import os


In [59]:
separator_regex = r'(,|\\t)'
# not working
def set_frame(item):
    # item is a path towards a file
    df = None
    filename, extension = os.path.splitext(item)
    match(extension):
        case '.csv':
            df = pd.read_csv(item)
        case '.parquet':
            df = pd.read_parquet(item)
        case '.xlsx':
            df = pd.read_excel(item)
        case '.html':
            df = pd.read_html(item)[0]
        case '.json':
            df = pd.read_json(item)
        case '.pickle' | '.pkl':
            df = pd.read_pickle(item)
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    return df
    
def append_files(df, file_list):
    file_list.sort()
    for item in file_list:
        filename, extension = os.path.splitext(item)
        ctr = 1
        match(extension):
            case '.csv':
                df = pd.concat([df, pd.read_csv(item, engine="python")], ignore_index=True)
            case '.parquet':
                df = pd.concat([df, pd.read_parquet(item)], ignore_index=True)
            case '.xlsx':
                df = pd.concat([df, pd.read_excel(item)], ignore_index=True)
            case '.html':
                df = pd.concat([df, pd.read_html(item)[0]], ignore_index=True)
            case '.json':
                df = pd.concat([df, pd.read_json(item)], ignore_index=True)
            case '.pickle' | '.pkl':
                df = pd.concat([df, pd.read_pickle(item)], ignore_index=True)
        print("Read",ctr,"file/s")
        ctr = ctr+1

    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    return df

In [61]:
campaign_data = glob.glob('C:/dev/dataset/Marketing Department/campaign_data*')

with open(campaign_data[0], 'r') as file:
  filedata = file.read()

# Replace the target string
filedata = filedata.replace('\t', ',')

# Write the file out again
with open(campaign_data[0], 'w') as file:  
    file.write(filedata)
    
df = set_frame(campaign_data[0])

if len(campaign_data) > 1:
    campaign_data.pop(0)
    df = append_files(df, campaign_data)

df.replace(to_replace={'discount': '[^0-9]'}, value="", inplace=True, regex=True)

display(df)


,campaign_id,campaign_name,campaign_description,discount
0,CAMPAIGN24776,wouldn't you know it,"""Twee retro vinyl single-origin coffee sartori...",1
1,CAMPAIGN33679,could be written on the back of a postage stamp,"""Fanny pack gentrify cardigan messenger bag."" ...",1
2,CAMPAIGN49972,me neither,"""DIY pug leggings everyday craft beer cardigan...",10
3,CAMPAIGN61872,on the huh,"""Trust fund pinterest chambray."" - Claude Aufd...",5
4,CAMPAIGN03110,stick a fork in it,"""YOLO tumblr Yuccie austin."" - Jordi Kunde",1
5,CAMPAIGN46302,you must be new here,"""Craft beer XOXO hella tacos chillwave cred or...",1
6,CAMPAIGN11190,mind your own beeswax,"""Street shoreditch viral before they sold out ...",10
7,CAMPAIGN53595,how do I get to the train station,"""Craft beer venmo lomo fixie readymade marfa.""...",20
8,CAMPAIGN29983,would it hurt,"""Vegan migas ramps keytar wolf cray kickstarte...",1
9,CAMPAIGN52447,pound for pound,"""Semiotics biodiesel everyday craft beer etsy ...",1


In [60]:
transactional_campaign_data = glob.glob('C:/dev/dataset/Marketing Department/transactional_campaign_data*')
df = set_frame(transactional_campaign_data[0])

if len(transactional_campaign_data) > 1:
    transactional_campaign_data.pop(0)
    df = append_files(df, transactional_campaign_data)

display(df)
# df.to_parquet('campaign_data.parquet')

,transaction_date,campaign_id,order_id,estimated arrival,availed
0,2023-08-14,CAMPAIGN49972,0612c246-57f1-40e8-9993-0f8d41992049,10days,1
1,2021-10-12,CAMPAIGN46302,b4c411de-2fd3-4806-91ae-165edc9baa12,13days,0
2,2023-01-30,CAMPAIGN29983,26de6b40-db2d-40b9-a64c-58736eaf0381,3days,1
3,2022-01-05,CAMPAIGN46302,26b60a4e-aafe-4b99-bace-034d088a4a53,8days,1
4,2023-09-19,CAMPAIGN46302,4aab29ae-e610-46bf-92af-199f6f420cee,14days,1
...,...,...,...,...,...
124882,2022-04-14,CAMPAIGN61872,6a5163c2-ba62-461c-a729-5b06fe266bd9,12days,0
124883,2022-11-23,CAMPAIGN11190,883e2fe2-bbdf-4b5e-8348-c2bcba07da0e,15days,1
124884,2023-07-28,CAMPAIGN33679,575b0a2d-5539-4565-a10b-1f4f6516cd27,12days,1
124885,2020-04-12,CAMPAIGN49972,ba506489-1505-481d-b0b1-8cdbf04a515f,6days,1
